# Preprocess


In [60]:
using Statistics, LinearAlgebra, Random, DataFrames, Plots, StatsPlots, CSV, JSON, Dates

In [61]:
data = CSV.read("movies_metadata.csv")

thread = 1 warning: only found 10 / 24 columns on data row: 19730. Filling remaining columns with `missing`
thread = 1 warning: only found 15 / 24 columns on data row: 19731. Filling remaining columns with `missing`
thread = 1 warning: only found 10 / 24 columns on data row: 29503. Filling remaining columns with `missing`
thread = 1 warning: only found 15 / 24 columns on data row: 29504. Filling remaining columns with `missing`
thread = 1 warning: only found 10 / 24 columns on data row: 35587. Filling remaining columns with `missing`
thread = 1 warning: only found 15 / 24 columns on data row: 35588. Filling remaining columns with `missing`


,adult,belongs_to_collection
,String,String⍰
1,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"
2,False,missing
3,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}"
4,False,missing
5,False,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}"
6,False,missing
7,False,missing
8,False,missing
9,False,missing


In [62]:
y = data[:, :vote_average]

45466-element Array{Union{Missing, Float64},1}:
 7.7
 6.9
 6.5
 6.1
 5.7
 7.7
 6.2
 5.4
 5.5
 6.6
 6.5
 5.7
 7.1
 ⋮  
 4.0
 6.0
 6.3
 7.0
 7.0
 3.5
 5.7
 4.0
 9.0
 3.8
 0.0
 0.0

In [63]:
names(data)

24-element Array{Symbol,1}:
 :adult                
 :belongs_to_collection
 :budget               
 :genres               
 :homepage             
 :id                   
 :imdb_id              
 :original_language    
 :original_title       
 :overview             
 :popularity           
 :poster_path          
 :production_companies 
 :production_countries 
 :release_date         
 :revenue              
 :runtime              
 :spoken_languages     
 :status               
 :tagline              
 :title                
 :video                
 :vote_average         
 :vote_count           

In [64]:
"Computes a onehot vector for every entry in column given a set of categories cats"
function onehot(column, cats=unique(column))
    result = zeros(size(column)[1], size(cats)[1])
    new_cats = Dict()
    for (n, f) in enumerate(cats)
       new_cats[f] = n
    end
    
    for i in 1:size(column)[1]
        index = get(new_cats, column[i], 0)
        if index != 0
            result[i, index] = 1
        end
    end
    result
end

onehot

In [65]:
"Computes a manyhot vector for every entry in an array of arrays given a set of categories cats"
function manyhot(column)
    cats = Set()
    for set in column
        for item in set
            push!(cats, item)
        end
    end
    cats = unique(cats)
    
    result = zeros(size(column)[1], size(cats)[1])
    new_cats = Dict()
    for (n, f) in enumerate(cats)
       new_cats[f] = n
    end
    
    for i in 1:size(column)[1]
        set_of_descriptions = Set(column[i])
        for description in set_of_descriptions
            index = get(new_cats, description, 0)
            if index != 0
                result[i, index] = 1
            end
        end
    end
    result
end

manyhot

### Preprocess genre using many-hot encoding

In [66]:
function preprocess_json(d)
    if ismissing(d) || d == "False"
        return Set()
    end
    x = d
    x = replace(x, "\"" => "\'")
    x = replace(x, "\\xa0" => " ")
    x = replace(x, "'name'" => "\"name\"")
    x = replace(x, "'id'" => "\"id\"")
    x = replace(x, ": '" => ": \"")
    x = replace(x, "'}" => "\"}")
    x = replace(x, "'," => "\",")
    x = replace(x, "Orlenok\"" => "Orlenok'")
    x = replace(x, "'iso_3166_1'" => "\"iso_3166_1\"")
    result = Set{String}()
    try
        for dic in JSON.parse(x)
            push!(result, dic["name"])
        end
    catch err
        println(d)
        println(x)
        println("=============================")
        return Set()
    end
    return result
end

preprocess_json (generic function with 1 method)

In [67]:
genres = manyhot(map(preprocess_json,data[:,:genres]))

45466×32 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  …  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  1

### Preprocess production company using many-hot encoding

In [68]:
production_companies = manyhot(map(preprocess_json,data[:,:production_companies]))

45466×23537 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0

There are so many production companies and we don't want to overfit our data, so we exclude it for now.

### Production countries using many-hot encoding

In [69]:
production_countries = manyhot(map(preprocess_json,data[:,:production_countries]))

4.3
4.3
6.0
6.0
7.0
7.0


45466×160 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  

### Preprocess popularity to float type

In [70]:
function parse_string_to_float(x)
    if ismissing(x)
        return 0
    end
    result = 0
    try
        result = parse(Float64, x)
    catch
        println(x)
        println("=========")
    end
    return result
end

parse_string_to_float (generic function with 1 method)

In [71]:
popularity = map(parse_string_to_float,data[:,:popularity])

Beware Of Frost Bites


45466-element Array{Real,1}:
 21.946943
 17.015539
 11.7129  
  3.859495
  8.387519
 17.924927
  6.677277
  2.561161
  5.23158 
 14.686036
  6.318445
  5.430331
 12.140733
  ⋮       
  0.139936
  0.225051
  0.222814
  0.076061
  0.38645 
  0.661558
  5.683753
  0.072051
  0.178241
  0.903007
  0.003503
  0.163015

### Preprocess budget with boolean encoding

In [72]:
budget = data[:,:budget]
Random.seed!(3)
randlist = filter(x -> x != "0", data[:,:budget])
rand_num = length(randlist)
budget_rep = replace(budget, "/ff9qCepilowshEtG2GYWwzt2bs4.jpg"=>"0")
budget_rep = replace(budget_rep, "/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg"=>"0")
budget_rep = replace(budget_rep, "/zaSf5OG7V8X8gqFvly88zDdRm46.jpg"=>"0")
budget_rep = replace(budget_rep, "0"=>randlist[rand(1:rand_num)], NaN=>randlist[rand(1:rand_num)])
budget_repint = parse.(Float64, string.(budget_rep)) / 10^6
n = length(budget_repint)
budget_feature = zeros(n, 2)
for r in 1:n
    if budget_repint[r] > 20
        budget_feature[r,:] = [1,1]
    elseif budget_repint[r] > 1
        budget_feature[r,:] = [1,0]
    else
        budget_feature[r,:] = [0,0]
    end
end

In [73]:
sum(budget_feature)/45466

1.8248141468349977

### Preprocess runtime using boolean encoding, change missing to mean value

In [74]:
time = data[:, :runtime]
valid = filter(x -> ! ismissing(x), time)
mean_val = mean(valid)
for i in 1:size(time, 1)
    if ismissing(time[i]) || time[i] == 0
        time[i] = mean_val
    end
end

min_val = minimum(time)
max_val = maximum(time)
q_val = quantile(time, [0.25, 0.5, 0.75])

runtime = zeros(size(time, 1), 3) # the new matrix with many code
for i in 1:size(time, 1)
    t = time[i]
    runtime[i, :] = Int.([t>=q_val[1], t>=q_val[2], t>=q_val[3]])
end
runtime

45466×3 Array{Float64,2}:
 0.0  0.0  0.0
 1.0  1.0  0.0
 1.0  1.0  0.0
 1.0  1.0  1.0
 1.0  1.0  0.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  0.0
 1.0  1.0  0.0
 1.0  1.0  1.0
 1.0  1.0  0.0
 1.0  0.0  0.0
 0.0  0.0  0.0
 ⋮            
 0.0  0.0  0.0
 1.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 1.0  1.0  0.0
 1.0  0.0  0.0
 1.0  1.0  1.0
 1.0  0.0  0.0
 1.0  0.0  0.0
 0.0  0.0  0.0

### Preprocess runtime using one-hot encoding on month/quarter

In [75]:
date = data[:, :release_date]
months = Int.(zeros(size(date, 1)))
for i in 1:size(date, 1)
    x = date[i]
    if ismissing(x)
        months[i] = rand(1:12,1)[1]
    else
        months[i] = Dates.month(x)
    end
end

release_date = zeros(size(months, 1), 4)
for i in 1:length(months)
    d = months[i]
    release_date[i, :] = Int.([d in 1:3, d in 4:6, d in 7:9, d in 10:12])
end
release_date

45466×4 Array{Float64,2}:
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0
 ⋮                 
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0
 0.0  1.0  0.0  0.0

# Experiment

In [76]:
y = map(z -> ismissing(z) ? 0 : z, y)

45466-element Array{Real,1}:
 7.7
 6.9
 6.5
 6.1
 5.7
 7.7
 6.2
 5.4
 5.5
 6.6
 6.5
 5.7
 7.1
 ⋮  
 4.0
 6.0
 6.3
 7.0
 7.0
 3.5
 5.7
 4.0
 9.0
 3.8
 0.0
 0.0

### Separate data into training and test dataset

In [100]:
function separate_dataset(y, dataset)
    new_dataset = hcat(y, dataset)
    new_dataset = new_dataset[shuffle(1:end), :]
    
    train_proportion = 0.8
    n = size(dataset, 1)
    println("Size of dataset: ", string(n))
    
    ntrain = convert(Int, round(train_proportion*n))

    target = new_dataset[:, 1]
    train_x = new_dataset[1:ntrain, 2:size(dataset, 2)]
    test_x = new_dataset[ntrain+1:n, 2:size(dataset, 2)]
    train_y = target[1:ntrain, :]
    test_y = target[ntrain+1:n, :]
    return train_x, train_y, test_x, test_y
end

separate_dataset (generic function with 1 method)

### Calculate least square

In [78]:
"""This function just computes the mean squared error."""
function MSE(y, pred)
    n = size(y)[1]
    err = 0
    for i in 1:n
        err += (y[i] - pred[i])^2
    end
    return err/n
end

"""This function plots the main diagonal; 
for a "predicted vs true" plot with perfect predictions,
all data lies on this line"""
function plotDiagonal(xmin, xmax)
    xsamples = [xmin, xmax]
    plot!(xsamples, xsamples, color=:black)
end

"""This helper funciton plots x vs, y and labels the axes."""
function plotdata(x,y,xname, yname; margin=.05, plotDiag=true, zeromin=false)
    scatter(x,y, label="data")
    xlabel!(xname)
    ylabel!(yname)
    range_y = maximum(y) - minimum(y)
    range_x = maximum(x) - minimum(x)
    if plotDiag
        plotDiagonal(minimum(x)-margin*range_x, maximum(x)+margin*range_x)
    end
    if zeromin
        ylims!((0.0,maximum(y)+margin*range_y))
        xlims!((0.0,maximum(x)+margin*range_x))
    else
        ylims!((minimum(y)-margin*range_y,maximum(y)+margin*range_y))
        xlims!((minimum(x)-margin*range_x,maximum(x)+margin*range_x))
    end
end

"""This function plots the predicted labels vs the actual labels
(We only plots the first 1000 points to avoid slow plots.)"""
function plot_pred_true(test_pred, test_y, max_points = 1000)
    plotdata(test_pred[1:max_points], test_y[1:max_points], "Predicted (\$)", "True (\$)", zeromin=true)
end

plot_pred_true

In [79]:
function calculate_MSE(train_x, train_y, test_x, test_y)
    # The weights of your linear equation (compute via least squares)
    w = train_x\train_y

    train_pred = train_x*w
    test_pred = test_x*w

    train_MSE = MSE(train_y, train_pred)
    test_MSE = MSE(test_y, test_pred)
    
    println("Train MSE\t", train_MSE)
    println("Test MSE \t", test_MSE)
    return test_pred
end

calculate_MSE (generic function with 1 method)

### First, let's try all the small features to avoid overfitting (exclude production countries and production companies)

In [101]:
dataset = hcat(ones(size(data,1)), budget_feature, popularity, release_date, runtime, genres)
train_x, train_y, test_x, test_y = separate_dataset(y, dataset)
test_pred = calculate_MSE(train_x, train_y, test_x, test_y)

Size of dataset: 45466
Train MSE	3.4831746186395947
Test MSE 	3.3538885691769478


9093×1 Array{Any,2}:
 5.556680582948648 
 6.259550551663693 
 5.469261530351277 
 6.008827495160706 
 4.743734409617135 
 5.729618579227725 
 6.225476294629596 
 6.052469234584286 
 5.666910661232216 
 5.110609582122738 
 5.494207214970569 
 5.294645265900871 
 5.146207765900871 
 ⋮                 
 5.831352665071385 
 4.939758376118801 
 6.156456516966938 
 5.7245883878154835
 4.9239581411263185
 5.411625805763454 
 4.6710099165384955
 5.773631965035326 
 5.2130143302354   
 5.086253374528389 
 5.626993082948648 
 5.301421401643715 

### Let's then try without genre which has 32 columns

In [102]:
dataset = hcat(ones(size(data,1)), budget_feature, popularity, release_date, runtime)
println(size(dataset))
train_x, train_y, test_x, test_y = separate_dataset(y, dataset)
test_pred = calculate_MSE(train_x, train_y, test_x, test_y)

(45466, 11)
Size of dataset: 45466
Train MSE	3.516125452018061
Test MSE 	3.5369140582835943


9093×1 Array{Any,2}:
 5.878927203562704
 5.730489703562704
 5.800802203562704
 6.339864703562704
 5.007460127236646
 5.753927203562704
 5.202772627236646
 5.847677203562704
 5.699239703562704
 5.328125         
 6.121114703562704
 5.312147627236646
 5.28125          
 ⋮                
 6.042989703562704
 5.917989703562704
 5.777364703562704
 5.257460127236646
 5.871114703562704
 5.257460127236646
 5.707052203562704
 6.285177203562704
 5.249647627236646
 5.155897627236646
 5.2421875        
 5.660177203562704

### Since the result with genres in the feature set does better than without, let's try adding production countries (160 columns)

In [109]:
dataset = hcat(ones(size(data,1)), production_countries)
println(size(dataset))
train_x, train_y, test_x, test_y = separate_dataset(y, dataset)
test_pred = calculate_MSE(train_x, train_y, test_x, test_y)

(45466, 161)
Size of dataset: 45466


SingularException: SingularException(160)

In [162]:
factorize(production_countries)

QRPivoted{Float64,Array{Float64,2}}
Q factor:
45466×45466 LinearAlgebra.QRPackedQ{Float64,Array{Float64,2}}:
 -0.00687565  -0.000809416  -0.000268892  …  -0.000239043  -1.05058e-5 
 -0.00687565  -0.000809416  -0.000268892     -7.51847e-5    0.0157091  
 -0.00687565  -0.000809416  -0.000268892     -1.19805e-5   -0.00154452 
 -0.00687565  -0.000809416  -0.000268892     -1.32786e-5   -0.00136451 
 -0.00687565  -0.000809416  -0.000268892      9.47434e-6   -0.000236983
 -0.00687565  -0.000809416  -0.000268892  …  -2.43122e-5   -0.000577174
 -0.00687565  -0.000809416  -0.000268892     -1.23647e-7   -0.000180694
 -0.00687565  -0.000809416  -0.000268892     -6.32187e-6   -0.000415827
 -0.00687565  -0.000809416  -0.000268892      0.0331256    -1.39416e-5 
 -0.00687565   0.0149273    -0.00180249       1.44981e-5   -0.000499399
 -0.00687565  -0.000809416  -0.000268892  …  -7.79179e-5   -3.80673e-5 
 -0.00687565  -0.000809416   0.0157582       -0.000205028  -0.000166957
 -0.00687565  -0.000809416 